In [9]:
from torchvision.datasets import STL10
from torchvision.transforms import ToTensor
from torchvision import models
import torch
from torch import nn
from src.model import Classifier
from src.utils import get_file

In [10]:
TRAIN_DATASET = STL10(root="data", split="train", download=False, transform=ToTensor())
TEST_DATASET = STL10(root="data", split="test", download=False, transform=ToTensor())

In [11]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [12]:
model = models.resnet18(pretrained=True)
pre_file = get_file('PZGY_finetuned.ckpt')
if pre_file is not None:
        cls = Classifier.load_from_checkpoint(pre_file, feature_extractor=model)
        model.load_state_dict(cls.model.state_dict())
model.fc = Identity()
model.eval()
labels = []
count = [0,0,0,0,0,0,0,0,0,0]
with torch.no_grad():
    for i in range(10):
        t = torch.zeros([500,512])
        labels.append(t)

    for xi, x in enumerate(TRAIN_DATASET):
        c, y = x
        cd = model(torch.unsqueeze(c, 0))
        labels[y][count[y]] = cd[0]
        count[y] += 1



In [14]:
label_std = []
label_mean = []
for i in range(10):
    label_std.append(torch.std(labels[i], dim=0))
    label_mean.append(torch.mean(labels[i], dim=0))

In [16]:
print(label_std[0].shape)

torch.Size([512])
